Carlos Garcia - 21000475

### K-Folds Cross Validation

**K-fold.** Es una técnica de validación cruzada, cuyo objetivo es reducir el "bias" que puede presentarse al momento de entrenar un modelo de machine learning. Esta técnica se basa en el principio de evaluar el modelo sobre data que no ha sido utilizada para su entrenamiento, particionando el set de datos de entrenamiento para que este pueda ser usado para entrenamientos y validaciones. Su característica principal se basa en poder generar **K** particiones de la data, en las cuales cada una de estas particiones es utilizada para evaluar el modelo.
De esta forma se entrenan sobre las K-1 y se evalua sobre la partición restante, repitiendo este proceso hasta recorrer todas las particiones generadas, los resultados obtenidos son promediados generando un estimador con menos sesgo para las métricas de evaluación a considerar.  

<img src='./imgs/kfolds.png'>

https://towardsdatascience.com/cross-validation-k-fold-vs-monte-carlo-e54df2fc179


En el proyecto de predicción de sobrevivientes del Titanic se trabajó con un split que generó sets de entrenamiento y pruebas. K-folds fue implementado al evitar particionar el dataset de entrenamiento en un set de validación adicional, es así como a partir del set de entrenamiento se trabajó con la función KFold de Scikit-learn la cual permitió particionar el dataset de entrenamiento K veces, de forma que se generaron predicciones sobre cada una de estas particiones y se promediaron los resultados. Siendo estos promedios los resultados guardados en la bitácora del proyecto, que posteriormente fueron utilizados para seleccionar los mejores modelos de cada tipo. 

### Support Vector Machines (SVM)

Es un algoritmo de Machine Learning que permite resolver problemas de clasificación lineal y no lineal, basado en la aplicación de vectores de soporte que determinan no solo una frontera de decisión si no también un margen de seperación para la tarea a realizar.

<img src="./imgs/svm_graph.png">

https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47

#### Hipótesis

La hipótesis usada en SVM es una función por partes o condicional definida a partir de la la expresión lineal. 

<img src="./imgs/svm_hypothesis.png">

*Hands-On Machine Learning with Scikit-Learn & Tensorflow*

#### Función de Costo

La función de costos de SVM se conoce como bisagra(hinge).
Esta función busca encontrar el punto mínimo de la curva en ciertos valores de otra curva. Es decir busca el punto mínimo a partir de una restricción que se relaciona con la función. (Optimización con restricciones)


La funcion hinge no es un promedio de los errores si no más bien una suma de estos. En esta funcion el parámetro de regularización es denotado con la letra *C* y presenta un efecto igual al recíproco del término de regularización (1/lambda). 


<img src="./imgs/svm_costfunction.png">

*Hands-On Machine Learning with Scikit-Learn & Tensorflow*

#### Propiedades y comparación con otros algoritmos

SVM es un algoritmo que, a diferencia de otros algoritmos (e.g. regresión logística), no define una frontera de decisión. Esto debido a que SVM es un algoritmo que computa a su vez vectores de soporte que buscan ampliar la brecha que separa una clase a la otra, permitiendo la configuración de este margen generando clasificasiones de margen suave y duro. 

**Ventajas**
- Este algoritmo puede ser utilizado tanto para problemas de clasificasción como para problemas de regresión
- Maneja problemas lineales y no lineales 
- La posibilidad de implementar kernel trick reduce significativamente la complejidad del modelo

**Desventajas**
- Si la data no esta escalada el modelo se ve altamente afectado
- La aplicación de distintos kernel requiere del conocimiento de los distintos hiperparámetros (e.g. hiperparámetro C y Gamma) 

#### Kernel trick

Kernel es una funcion que mide la similitud entre dos vectores o funciones. Su aplicación permite generar un espacio latente, el cual es un espacio diferente obtenido a partir de ciertas transformaciones de los datos. 

El truco del kernel (kernel trick) es un concepto basado en *basis expansion* que permite replicar los efectos de transformar varias características sin tener que agregarlas realmente, evitando así el impacto computacional que genera el agregar una cierta cantidad de términos polinomiales. Esto con el objetivo de transformar las características en cuestión y generar un dataset separable para propósitos de la clasificación. 

En esencia el Kernel Trick se basa en ahorrar las transformacion de las características, de forma que el kernel es capaz de computar el producto punto de vectores transformados basado unicamente en los vectores originales, obviando así las transformaciones que serían necesarias de aplicar (phi). 

**Kernel trick**

<img src="./imgs/kernel_trick.png">

*https://medium.com/@sonalij663/what-is-kernel-tricks-bb176291e60f*

**Kernels**

<img src="./imgs/svm_kernel.png">

*Hands-On Machine Learning with Scikit-Learn & Tensorflow*

### Algoritmo de aprendizaje modelo SVM

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from auxiliaryFunctions import getMetrics
from datetime import datetime
import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


#### Datos

In [3]:
data = pd.read_csv('./input/trainset_svm_essay.csv', index_col = 0)
data.head()

,SibSp,Parch,Fare,passenger_class,isFemale,passenger_survived
329,-0.460103,0.829588,0.490420,1.599696,1,1
749,-0.460103,-0.474312,-0.472168,-0.815964,0,0
203,-0.460103,-0.474312,-0.482229,-0.815964,0,0
421,-0.460103,-0.474312,-0.472488,-0.815964,0,0
97,-0.460103,0.829588,0.593505,1.599696,0,1


#### Entrenamiento

In [4]:
def trainModel(x, y, epochs = 100, batch_size = 1, lr = 0.01, c = 0, kprint = 10):
    
    #Get total iterations
    iters = x.shape[0] // batch_size

    #Initialize the graph
    tf.reset_default_graph()

    #Defining tensors and variables
    X = tf.placeholder(dtype = tf.float32, shape = [None, x.shape[1] + 1], name = 'features')
    labels = tf.placeholder(dtype = tf.float32, shape = [None, 1], name = 'class_labels')
    W = tf.get_variable(name = 'weights', shape = [x.shape[1] + 1, 1], dtype = tf.float32, initializer = tf.zeros_initializer())

    #Estimating values
    logits = tf.matmul(X, W, name = 'logit')

    #Calculating cost function (hinge)
    with tf.name_scope('cost_function'):
        margin_term = tf.multiply(tf.constant(1/2), tf.reduce_sum(tf.pow(W, 2)), name = 'margin_term')
        hinge = tf.losses.hinge_loss(labels = labels, logits = logits)
        penalizing_term = tf.multiply(c, hinge, name = 'penalizing_term')
        loss = tf.add(margin_term, penalizing_term, name = 'loss')

    #Tensorboard scalar summary
    loss_summary = tf.summary.scalar(name = 'Hinge_Loss', tensor = loss)

    #Calculate accuracy
    with tf.name_scope('accuracy_definition'):
        preds = tf.sign(logits)
        t = tf.subtract(tf.multiply(labels, 2), tf.constant(1.0), name = 't_vector')
        accuracy_tensor = tf.reduce_mean(tf.cast(tf.equal(preds, t),tf.float32), name = 'accuracy_metric')

    #Tensorboard scalar accuracy
    accuracy_summary = tf.summary.scalar(name = 'Accuracy', tensor = accuracy_tensor)

    #Gradient and cost error optimization
    with tf.name_scope('optimize_loss'):
        gradients = tf.gradients(loss, [W], name = 'gradients')
        optimizer = tf.assign(W, W - lr * gradients[0], name = 'optimizer')


    with tf.Session() as session:

        #Initialize global vars
        session.run(tf.global_variables_initializer())

        #Reshaping data
        ones = np.expand_dims(np.ones_like(x[:,0]), axis = 1)
        x = np.hstack((ones, x))
        y = np.expand_dims(y, axis = 1)

        #Whole batch dictionary
        feed_dict_model = {X:x, labels:y}

        #Define tensorboard writer
        dt_string = datetime.now().strftime("%Y%m%d_%H%M")
        writer = tf.summary.FileWriter('./graphs/svm/{}_svm_epochs={}_mbatch={}_lr={}_c={}_minmax'.format(dt_string, epochs, batch_size, lr, c), session.graph)

        
        for epoch in range(0, epochs):
            for i in range(0, iters):
                start_sample = i * batch_size
                end_sample = start_sample + batch_size
                x_mb = x[start_sample:end_sample]
                y_mb = y[start_sample:end_sample]

                feed_dict = {X:x_mb, labels:y_mb}
                _, weights = session.run([optimizer, W], feed_dict = feed_dict)

            predictions = session.run([preds], feed_dict = feed_dict_model)
            cost, lsummary = session.run([loss, loss_summary], feed_dict = feed_dict_model)
            writer.add_summary(lsummary, epoch + 1)

            tvalue, accuracy, asummary = session.run([t, accuracy_tensor, accuracy_summary], feed_dict = feed_dict_model)
            writer.add_summary(asummary, epoch + 1)

            if (epoch + 1) % kprint == 0:
                print("Epoch {}: HingeLoss={} --- Accuracy={}".format(epoch + 1, cost, accuracy))
        

        #Calculate final model metrics
        _, weights = session.run([optimizer, W], feed_dict = feed_dict_model)
        predictions, cost = session.run([preds, loss], feed_dict = feed_dict_model)
        tvalue, accuracy = session.run([t, accuracy_tensor], feed_dict = feed_dict_model)
        print("Final model: HingeLoss={} --- Accuracy={}".format(cost, accuracy))

        #Close tensorboard writer
        writer.close()

        return weights

In [5]:
weights = trainModel(data.iloc[:,:-1].values, data.iloc[:,-1].values, epochs = 1000, batch_size = 200, lr = 0.1, c = 1.1, kprint = 50)

Epoch 50: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 100: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 150: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 200: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 250: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 300: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 350: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 400: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 450: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 500: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 550: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 600: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 650: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 700: HingeLoss=0.9643999338150024 --- Accuracy=0.699438214302063
Epoch 7

#### Grafo SVM

<img src="./imgs/svm_grafo.png">

#### Métricas en Tensorboard

<img src="./imgs/hinge_loss.png">

<img src="./imgs/svm_accuracy.png">

#### Predicciones

In [6]:
testset = pd.read_csv('./input/testset_svm_essay.csv', index_col = 0)
testset.head()

,SibSp,Parch,Fare,passenger_class,isFemale,passenger_survived
172,0.414308,0.582097,-0.534701,-0.874341,1,1
524,-0.536476,-0.481121,-0.637342,-0.874341,0,0
452,-0.536476,-0.481121,-0.097841,1.444281,0,0
170,-0.536476,-0.481121,0.053329,1.444281,0,0
620,0.414308,-0.481121,-0.447393,-0.874341,0,0


In [7]:
def getPredictions(x, weights):
    ones = np.expand_dims(np.ones_like(x[:,0]), axis = 1)
    x = np.hstack((ones, x))
    preds = np.sign(np.matmul(x, weights))
    
    return preds

In [8]:
y_labels = testset.iloc[:, -1].values
y_labels = np.piecewise( y_labels, [y_labels == 0, y_labels > 0], [-1, 1])
y_labels = np.expand_dims(y_labels, axis = 1)

In [9]:
preds = getPredictions(testset.iloc[:, :-1].values, weights)

In [10]:
metrics = getMetrics('tensorflow_SVM', y_labels, preds)
metrics

,model,accuracy,error,precision,recall,f1-score
0,tensorflow_SVM,0.7207,0.2793,0.6364,0.5385,0.5833
